Importing the basic libraries for preprocessing...

In [1]:
import numpy as np
import os
import re
#from math import *
from numpy.linalg import norm
import gensim
from gensim.models import Word2Vec

C:\Users\nidhi1\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Setting up NN MODEL constants...

In [5]:
EMBEDDING_DIM = 100
BATCH_SIZE = 25
MAX_SENTENCE_LENGTH = 60

Reading the Input text file containing OS notes for training Word2vec model

In [10]:
sentences = []
os.chdir("C:\\NIDHI\\BITS\\exp")
with open('os_notes.txt') as f:
    for line in f:
        print(line)
        tokens = gensim.utils.simple_preprocess(line)
        print(tokens)
        sentences.append(tokens)

An operating system is a program that manages the computer hardware.

['an', 'operating', 'system', 'is', 'program', 'that', 'manages', 'the', 'computer', 'hardware']
It also provides a basis for application programs and acts as an intermediary between the computer user and the computer hardware.

['it', 'also', 'provides', 'basis', 'for', 'application', 'programs', 'and', 'acts', 'as', 'an', 'intermediary', 'between', 'the', 'computer', 'user', 'and', 'the', 'computer', 'hardware']
There are two Operating system goals Execute user programs and make solving user problems easier.

['there', 'are', 'two', 'operating', 'system', 'goals', 'execute', 'user', 'programs', 'and', 'make', 'solving', 'user', 'problems', 'easier']
An operating system Make the computer system convenient to use.

['an', 'operating', 'system', 'make', 'the', 'computer', 'system', 'convenient', 'to', 'use']
It handles multiple computer resources such as CPU, Internal/External memory, Processes, Tasks, Applications, U

Training the word2vec model....

In [7]:
w2v_model = Word2Vec(sentences,min_count=1)
nbwords = len(w2v_model.wv.index2word)
print(nbwords)
# the syn0 weight matrix in gensim corresponds to the Embedding matrix in Keras. 
# Importing the matrix directly and n ot trainig these weights as part of NN parameter training
weightMatrix = w2v_model.wv.syn0
vocab = dict([(k,w.index) for k,w in w2v_model.wv.vocab.items()])

394


Embedding matrix contains 394 words and each vector is of dim 100. Therefore size of weightMatrix is 290*100. We will use this w2v trained weight matrix as an input to the embedding layer of the NN

In [8]:
# Vertically stacking padding layer or mask layer with all zeros in the weightMatrix
weightMatrixWithPadding = np.vstack([np.zeros(EMBEDDING_DIM),weightMatrix])
weightMatrixWithPadding.shape

(395, 100)

Preparing the y vector using cosine similarity measure between modal answer and student answer. We will use y = 1 if cosine similarity returns more than 0.5 else y = 0. This helps to train the neural network for answers that are similar to the modal answer.

In [18]:
# defining a function to make the two sentence vectors of equal length i.e. padding faculty given
# modal answer/ student answer to make them of equal length

# obtaining vector embeddings for a given word using word2vec model defined above
def getWord2Vec(model,q):
    sentenceVector = []
    tokens = gensim.utils.simple_preprocess(q)
    lengthSent = (len(tokens))
    ##find its length and pad with arb token toreach max_Sent_length for uniformity
    for word in tokens:
        #print (word)
        try:
            wordVec = model[word]
            #print (wordVec)
        except:
            continue
        if not np.isnan(np.sum(wordVec)):
            sentenceVector.append(wordVec)
    if not sentenceVector:
        print("returning vector of zeros...")
        return np.zeros(EMBEDDING_DIM)
    #paddingEqualWidthSentence(sentenceVector,MAX_SENTENCE_LENGTH-lengthSent)
    sentenceVector = np.array(sentenceVector)
    return sentenceVector.mean(axis=0)

# computing cosine similarity between two given vectors
def similarity(v1,v2):
        #print (v1)
        #print (v2)
        angles = (np.dot(v1,v2)/(norm(v1,2)*norm(v2,2)))
        #print ("angles : ",angles)
        return angles


Defining the modal answer and obtaining its w2v embedding ...

In [19]:
modal_ans = "An operating system acts as an interface between the user and computer hardware. Operating system controls and coordinates the use of computing resources among the various application programs. Some examples of operating systems are windows, linux, unix, android and ios."
re.sub('operating system','',modal_ans)
#re.sub('system','',modal_ans)
ansvector = getWord2Vec(w2v_model,modal_ans)
ansvector.shape

(100,)

In [20]:
st_ans = "An operating system acts as an interface between the user and computer hardware. Operating system controls and coordinates the use of computing resources among the various application programs. Some examples of operating systems are windows, linux, unix, android and ios."
re.sub('operating system','',st_ans)
#re.sub('system','',st_ans)
stvector = getWord2Vec(w2v_model,st_ans)
np.array_equal(ansvector, stvector)
similarity(ansvector,stvector)

0.99999994

Computing similarity scores or the desired outcome y for training...

In [21]:
cosineSim_Score = np.zeros(shape=(BATCH_SIZE,))
#embedding_matrix = np.zeros(shape=(NUM_ANS,EMBEDDING_DIM))
student_answers = []
student_answer_vector = []
i=0
with open('midSem.txt') as f:
    for studans in f:
        print(studans)
        studansvector = getWord2Vec(w2v_model,studans)
        print(studansvector.shape)
        #student_answers.append(studans)
        score = similarity(ansvector,studansvector)
        print(score)
        if score>0.05:
            cosineSim_Score[i] = 1
        else:
            cosineSim_Score[i] = 0 
        student_answers.insert(i,studans)
        student_answer_vector.insert(i,studansvector)
        i = i+1
        # print(studans)
        # print(rank)

Operating system acts as intermediary between hardware and software.

(100,)
0.905177
full form of OS is operating system.

(100,)
0.876677
Operating system acts as an interface between user programs and computing resources.

(100,)
0.918759
The purpose of operating system is to provide an abstraction for execution of user programs from computer hardware.

(100,)
0.947035
OS is the most essential component of any computing environment.

(100,)
0.827449
A system which provides the interface between user and hardware.

(100,)
0.918943
the operating system are interface user and hardware are known as the operating system.

(100,)
0.960316
Operating system is an interface between user and hardware.

(100,)
0.918698
Operating system is an interface between user and hardware of a computer system. It helps user to make computer system do several jobs like processing, communicating etc.

(100,)
0.942837
Operating system is an interface between the user and the hardware. It is the system softwa

C:\Users\nidhi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
print(cosineSim_Score)

In [ ]:
# setting up correct cosine similarity measures
cosineSim_Score[0] = 1
cosineSim_Score[2] = 1
cosineSim_Score[3] = 1
cosineSim_Score[4] = 0
cosineSim_Score[5] = 0
cosineSim_Score[6] = 0
cosineSim_Score[7] = 0
cosineSim_Score[8] = 1
cosineSim_Score[9] = 1
cosineSim_Score[10] = 0
cosineSim_Score[11] = 0
cosineSim_Score[12] = 0
cosineSim_Score[13] = 0
cosineSim_Score[14] = 1
cosineSim_Score[15] = 1
cosineSim_Score[16] = 1
cosineSim_Score[17] = 0
cosineSim_Score[18] = 0
cosineSim_Score[19] = 0

Importing Keras libraries for defining the Siamese arch based LSTM NN

In [ ]:
from keras.layers import Embedding, Input, Dot, Dense, LSTM, Dropout, Activation
from keras.models import Model
from keras import backend as K
from keras.preprocessing import sequence, text

In [ ]:
input_seq_modal_ans = Input(shape=(60,))
input_seq_modal_ans.shape

In [ ]:
embedding_modal_ans = Embedding(nbwords+1, EMBEDDING_DIM, weights=[weightMatrixWithPadding], mask_zero=True, trainable=False)(input_seq_modal_ans)

In [ ]:
type(embedding_modal_ans)
embedding_modal_ans.shape

In [ ]:
lstm_output_modal_ans = LSTM(MAX_SENTENCE_LENGTH, return_sequences=False, name='modal_ans_lstm')(embedding_modal_ans)

In [ ]:
lstm_output_modal_ans.shape

Preparing model input for student answer

In [ ]:
input_seq_stud_ans = Input(shape=(60,))
embedding_stud_ans = Embedding(nbwords+1, EMBEDDING_DIM, weights=[weightMatrixWithPadding], mask_zero=True, trainable=True)(input_seq_stud_ans)
lstm_output_stud_ans = LSTM(MAX_SENTENCE_LENGTH, return_sequences=False, name='stud_ans_lstm')(embedding_stud_ans)

Merging the lstm layer output for student answer and modal answer using cosine similarity measure which is available using the DOT merge function

In [ ]:
merge_lstms = Dot(axes=1)([lstm_output_modal_ans,lstm_output_stud_ans])
type(merge_lstms)
merge_lstms.shape

In [ ]:
predictions = Dense(1, activation = 'sigmoid')(merge_lstms)
# defining the modalfor last network layer that computes Logistic Regression
finalModel = Model(inputs = [input_seq_modal_ans, input_seq_stud_ans], output=predictions)

In [ ]:
# Compiling the model
finalModel.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

Prepare input data in desired format....

Defining a function to convert text into sequence of numbers as is the requirement of EMbedding layer...

In [ ]:
def pad_sequences(sequence):
    dim = 60
    length = len(sequence)
    newseq = []
    count = dim-length
    for i in range(count):
        newseq.append('0')
        
    # we need to append dim-len zeros and then concatenate with sequence (input)
    newseq = sum([newseq,sequence],[])
    return newseq

In [ ]:
def text_to_sequence(string):
    seq = [] # np.array(np.zeros(MAX_SENTENCE_LENGTH))
   # if(string==""):
    
    #print("Function expects a string paramater. Please provide one")
    #else:
    s = string.lower().split(" ")
    for word in s:
        if word in vocab:
            seq.append(vocab[word])
                
    fseq = pad_sequences(seq)
    return fseq       

In [ ]:
res = text_to_sequence("An operating system is an interface")

In [ ]:
# res

In [ ]:
right_input = np.zeros(shape=(BATCH_SIZE, MAX_SENTENCE_LENGTH))
for i in range(BATCH_SIZE):
    res = text_to_sequence(student_answers[i])
    right_input[i] = res
  

    

In [ ]:
left_input = np.zeros(shape=(BATCH_SIZE, MAX_SENTENCE_LENGTH))
x_left = text_to_sequence(modal_ans)
for i in range(BATCH_SIZE):
    left_input[i] = x_left


In [ ]:
# verbose: 0 for no logging to stdout, 1 for progress bar logging, 2 for one log line per epoch.
# 

In [ ]:
finalModel.fit([left_input,right_input], y=cosineSim_Score, epochs=50, batch_size=BATCH_SIZE, verbose=2, validation_split=0.2)

In [ ]:
sans = "a software is an interface between operating system and hardware"
seq_sans = np.array([text_to_sequence(sans)])
#seq_sans.reshape(MAX_SENTENCE_LENGTH,)

In [ ]:
print(seq_sans.shape)
modal_ans_seq = np.array([x_left]) #.reshape(MAX_SENTENCE_LENGTH,)


In [ ]:
score = finalModel.predict(x=[modal_ans_seq,seq_sans], batch_size=1,verbose=1)

In [ ]:
sans2 = "a process is program under execution"
seq_sans2 = np.array([text_to_sequence(sans2)])

In [ ]:
score = finalModel.predict(x=[modal_ans_seq,seq_sans2], batch_size=1,verbose=1)